In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#read genes gtf file

genes_path='path-to-your/genes.gtf'
column_names = ["chr", "source", "feature", "start", "end", "score", "strand", "frame", "attribute"]
genes = pd.read_csv(genes_path, sep='\t', comment='#', header=None, names=column_names)
genes

,chr,source,feature,start,end,score,strand,frame,attribute
0,chr1,HAVANA,gene,11869,14409,.,+,.,"gene_id ""ENSG00000223972.5""; gene_type ""transc..."
1,chr1,HAVANA,transcript,11869,14409,.,+,.,"gene_id ""ENSG00000223972.5""; transcript_id ""EN..."
2,chr1,HAVANA,exon,11869,12227,.,+,.,"gene_id ""ENSG00000223972.5""; transcript_id ""EN..."
3,chr1,HAVANA,exon,12613,12721,.,+,.,"gene_id ""ENSG00000223972.5""; transcript_id ""EN..."
4,chr1,HAVANA,exon,13221,14409,.,+,.,"gene_id ""ENSG00000223972.5""; transcript_id ""EN..."
...,...,...,...,...,...,...,...,...,...
2742729,KI270744.1,ENSEMBL,transcript,51009,51114,.,-,.,"gene_id ""ENSG00000278625.1""; transcript_id ""EN..."
2742730,KI270744.1,ENSEMBL,exon,51009,51114,.,-,.,"gene_id ""ENSG00000278625.1""; transcript_id ""EN..."
2742731,KI270750.1,ENSEMBL,gene,148668,148843,.,+,.,"gene_id ""ENSG00000277374.1""; gene_type ""snRNA""..."
2742732,KI270750.1,ENSEMBL,transcript,148668,148843,.,+,.,"gene_id ""ENSG00000277374.1""; transcript_id ""EN..."


In [4]:
#select only gene features and extract gene_id from attributes

genes['GeneID'] = genes['attribute'].apply(lambda x: next((entry.split(' ')[1].strip('"') for entry in x.split(';') if entry.strip().startswith('gene_id')), None))
genes = genes[['chr', 'feature', 'start', 'end', 'strand', 'GeneID']]
genes = genes[genes['feature'] == 'gene']
genes

,chr,feature,start,end,strand,GeneID
0,chr1,gene,11869,14409,+,ENSG00000223972.5
12,chr1,gene,14404,29570,-,ENSG00000227232.5
25,chr1,gene,17369,17436,-,ENSG00000278267.1
28,chr1,gene,29554,31109,+,ENSG00000243485.5
36,chr1,gene,30366,30503,+,ENSG00000284332.1
...,...,...,...,...,...,...
2742636,KI270734.1,gene,72411,74814,+,ENSG00000276017.1
2742643,KI270734.1,gene,131494,137392,+,ENSG00000278817.1
2742659,KI270734.1,gene,138082,161852,-,ENSG00000277196.4
2742728,KI270744.1,gene,51009,51114,-,ENSG00000278625.1


In [6]:
#check for duplicate gene IDs

genes['GeneID'].duplicated().value_counts()

False    58780
Name: GeneID, dtype: int64

In [8]:
#select only standard chromosomes

chroms = ['chr'+str(x) for x in range(1,23)] + ['chrX', 'chrY']
genes = genes[(genes.chr.isin(chroms))]
genes

,chr,feature,start,end,strand,GeneID
0,chr1,gene,11869,14409,+,ENSG00000223972.5
12,chr1,gene,14404,29570,-,ENSG00000227232.5
25,chr1,gene,17369,17436,-,ENSG00000278267.1
28,chr1,gene,29554,31109,+,ENSG00000243485.5
36,chr1,gene,30366,30503,+,ENSG00000284332.1
...,...,...,...,...,...,...
2741715,chrY,gene,57184101,57197337,+,ENSG00000124334.17_PAR_Y
2741770,chrY,gene,57190738,57208756,+,ENSG00000270726.6_PAR_Y
2741776,chrY,gene,57201143,57203357,-,ENSG00000185203.12_PAR_Y
2741780,chrY,gene,57207346,57212230,+,ENSG00000182484.15_PAR_Y


In [9]:
#filter for Ensembl gene IDs only

genes = genes[genes['GeneID'].str.contains(r'^ENSG\d+\.\d+$')]
genes

,chr,feature,start,end,strand,GeneID
0,chr1,gene,11869,14409,+,ENSG00000223972.5
12,chr1,gene,14404,29570,-,ENSG00000227232.5
25,chr1,gene,17369,17436,-,ENSG00000278267.1
28,chr1,gene,29554,31109,+,ENSG00000243485.5
36,chr1,gene,30366,30503,+,ENSG00000284332.1
...,...,...,...,...,...,...
2741568,chrY,gene,26549425,26549743,+,ENSG00000224240.1
2741571,chrY,gene,26586642,26591601,-,ENSG00000227629.1
2741576,chrY,gene,26594851,26634652,-,ENSG00000237917.1
2741591,chrY,gene,26626520,26627159,-,ENSG00000231514.1


In [ ]:
genes[['GeneID', 'chr', 'start', 'end', 'strand']].to_csv('genes.saf', index=False, header=False, sep='\t') #this file is provided in the current directory

In [ ]:
#load the filtered bam files output from snakepipes mRNA-seq pipeline

bam = "path-to-directory/filtered_bam" #not provided here because too large

In [ ]:
# see https:/bioconductor.org/packages/release/bioc/vignettes/Rsubread/inst/doc/SubreadUsersGuide.pdf
# download from https:/rnnh.github.io/bioinfo-notebook/docs/featureCounts.html

!featureCounts -F SAF \
--primary \
-p -B \
-Q 10 \
-s 0 \
-T 8 \
--tmpDir [path-to-directory-to-store-temporary-files] \
-a genes.saf \
-o [path-to-directory-to-store-output-files]/all_featureCounts.txt \
[path-to-directory]/filtered_bam/sample1.filtered.bam [path-to-directory]/filtered_bam/sample2.filtered.bam [path-to-directory]/filtered_bam/sampleN.filtered.bam

# -F creates annotation index
# --primary only primary alignments will be counted
# -p is pairedEnd
# -B is only fragments that have both ends successfully aligned will be considered for summarization
# -Q minimum mapping quality score
# -s 0 means un-stranded (KAPA kit isn't stranded enough)
# -T number of threads


        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.0.6

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 8 BAM files                                      ||
||                                                                            ||
||                           LM124.filtered.bam                               ||
||                           LM125.filtered.bam                               ||
||                           LM126.filtered.bam                  

In [ ]:
#load the output files from the previous command

counts = pd.read_table('../data/all_featureCounts.txt', comment='#') 
counts.columns = [
    "Geneid", 
    "Chr", 
    "Start", 
    "End", 
    "Strand", 
    "Length", 
    "control_1", 
    "control_2", 
    "SRRM2depl_1", 
    "SRRM2depl_2", 
    "SONdepl_1", 
    "SONdepl_2", 
    "doubledepl_1", 
    "doubledepl_2", 
]
counts

,Geneid,control_1,control_2,SRRM2depl_1,SRRM2depl_2,SONdepl_1,SONdepl_2,doubledepl_1,doubledepl_2
0,ENSG00000223972.5,1,1,0,1,1,3,3,4
1,ENSG00000227232.5,149,103,86,88,91,53,73,73
2,ENSG00000278267.1,0,0,0,0,0,0,0,0
3,ENSG00000243485.5,0,2,2,2,2,0,1,2
4,ENSG00000284332.1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
58634,ENSG00000224240.1,0,0,0,0,0,0,0,0
58635,ENSG00000227629.1,0,0,0,0,0,0,0,0
58636,ENSG00000237917.1,0,0,0,4,0,0,0,0
58637,ENSG00000231514.1,0,0,0,0,0,0,0,0


In [12]:
# save count tables for different comparisons

counts_SON = counts[['Geneid',"control_1", "control_2", "SONdepl_1", "SONdepl_2"]] 
counts_SRRM2 = counts[['Geneid', "control_1", "control_2", "SRRM2depl_1", "SRRM2depl_2"]]
counts_double = counts[['Geneid', "control_1", "control_2", "doubledepl_1", "doubledepl_2"]] 
counts_all = counts[['Geneid', "control_1", "control_2", "SRRM2depl_1", "SRRM2depl_2", "SONdepl_1", "SONdepl_2", "doubledepl_1", "doubledepl_2"]] 

counts_SON.to_csv("../data/SONdepl_featureCounts.txt", sep='\t', index=False)
counts_SRRM2.to_csv("../data/SRRM2depl_featureCounts.txt", sep='\t', index=False)
counts_double.to_csv("../data/Doubledepl_featureCounts.txt", sep='\t', index=False)
counts_all.to_csv("../data/all_featureCounts.txt", sep='\t', index=False)